함수 데커레이터는 소스 코드에 있는 함수를 '표시'해서 함수의 작동을 개선할 수 있게 해준다. 이를 위해 다음을 살펴본다.

* 파이썬이 데커레이터 구문을 평가하는 방식
* 변수가 지역 변수인지 파이썬이 판단하는 방식
* 클로저의 존재 이유와 작동 방식
* `nonlocal`로 해결할 수 있는 문제

이후 다음과 같은 주제를 다룬다.

* 잘 작동하는 데커레이터 구현하기
* 표준 라이브러리에서 제공하는 재미있는 데커레이터들
* 매개변수화된 데커레이터 구현하기

# 7.1 데커레이터 기본 지식

데커레이터는 다른 함수를 인수로 받는 콜러블(데커레이트된 함수)이다. 데커레이터는 데커레이트된 함수에 어떤 처리를 수행하고, 함수를 반환하거나 함수를 다른 함수나 콜러블 객체로 대체한다. 데커레이터는 다음과 같이 작동한다.

```python
@decorate
def target():
    print('running target()')
```

위 코드는 다음 코드와 동일하게 작동한다.

```python
def target():
    print('running target()')
    
target = decorate(target)
```

두 코드를 실행한 후 `target`은 원래의 `target()` 함수를 가리키는 것이 아니라 `decorate(target)`이 반환한 함수를 가리키게 된다.

In [2]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [5]:
@deco
def target():
    print("running target()")
    
target() # 실제로는 inner()를 실행한다.

running inner()


In [8]:
target # inner()를 가리키고 있음

<function __main__.deco.<locals>.inner()>

요약하면, 
* 데커레이터는 데커레이트된 함수를 다른 함수로 대체한다.
* 데커레이터는 모듈이 로딩될 때 바로 실행된다.

# 7.2 파이썬이 데커레이터를 실행하는 시점

데커레이터의 핵심 특징은 데커레이트된 함수가 정의된 직후, 즉 일반적으로 파이썬이 모듈을 로딩하는 시점에 실행된다는 것이다.

In [15]:
!cat registration.py

registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1() # 이 경우 register()가 원래 함수를 반환하므로 f1()이 실행됨
    f2()
    f3()

if __name__=='__main__':
    main()


In [14]:
!python registration.py

running register(<function f1 at 0x7f4e78a3c0d0>)
running register(<function f2 at 0x7f4e76b6d6a8>)
running main()
registry -> [<function f1 at 0x7f4e78a3c0d0>, <function f2 at 0x7f4e76b6d6a8>]
running f1()
running f2()
running f3()


`register()`는 모듈 내의 다른 어떠한 함수보다 먼저 실행된다. 모듈이 로딩된 후 `registry`는 데커레이트된 두 개의 함수 `f1()`과 `f2()`에 대한 참조를 가진다. 이 두 함수는 `main()`에 의해 명시적으로 호출될 때만 실행된다.

In [18]:
import registration # 임포트 순간에 데커레이터가 실행됨

In [17]:
registration.registry

[<function registration.f1()>, <function registration.f2()>]

이 예제는 `임포트 타임`과 `런타임`의 차이를 명확히 보여준다.

# 7.3 데커레이터로 개선한 전략 패턴

데커레이터는 6.1절에서 구현한 전자상거래 프로모션 할인 코드를 개선하는데 유용하게 사용할 수 있다.

전자상거래 예제의 가장 큰 문제는 함수를 정의할 때, 그리고 가장 큰 할인 방식을 결정하는 `best_promo()` 함수에 의해 사용되는 `promos` 리스트에 함수명을 반복해서 사용한다는 점이다. 이를 데커레이터를 이용해서 다음과 같이 해결할 수 있다.

In [21]:
promos = []

def promotion(promo_func):
    """promotion() 데커레이터는 promo_func를 promos 리스트에 추가한 후 그대로 반환"""
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity_promo(order):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order): 
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

@promotion
def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

def best_promo(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)

이 방법은 다음과 같은 장점을 가진다.

* 프로모션 전략 함수명이 특별한 형태로 되어 있을 필요가 없다.
* `@promotion` 데커레이터는 임시로 어떤 프로모션을 배제할 수 있다.
* 프로모션 할인 전략을 구현한 함수는 `@promotion` 데커레이터가 적용되는 한 어느 모듈에서든 정의할 수 있다.

# 7.4 변수 범위 규칙

다음 예제에서는 함수 매개변수로 정의된 지역 변수 a와 함수 내부에 정의되지 않은 변수 b 등 두 개의 변수를 읽는 함수를 정의하고 테스트한다.

In [22]:
def f1(a):
    print(a)
    print(b)
    
f1(3)

3


NameError: name 'b' is not defined

In [23]:
b=6
f1(3)

3
6


In [26]:
b = 6
def f2(a):
    print(a)
    print(b)
    b=9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

이는 파이썬이 함수 본체를 컴파일할 때 b가 함수 안에서 할당되므로 b를 지역 변수로 판단하기 때문이다. 함수 안에 할당하는 문장이 있지만 인터프리터가 b를 전역 변수로 다루기 원한다면 `global` 키워드를 이용하여 선언해야 한다.

In [30]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
    
f3(3)

3
6


In [31]:
b

9

In [32]:
f3(3)

3
9


In [33]:
b=30
f3(3)

3
30


`dis` 모듈을 사용하면 파이썬 함수를 쉽게 바이트코드로 디스어셈블할 수 있다.

In [34]:
from dis import dis
dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [36]:
dis(f2) # b를 지역 변수에서 불러옴

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


# 7.5 클로저

클로저는 함수 본체에서 정의하지 않고 참조하는 비전역(nonglobal) 변수를 포함한 확장 범위를 가진 함수다. 예제를 통해 알아보자.

In [37]:
class Averager():
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [38]:
avg = Averager()
avg(10)

10.0

In [39]:
avg(11)

10.5

In [40]:
avg(12)

11.0

고위 함수 `make_averager()`를 이용해서 구현한 예제를 보자.

In [44]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

In [45]:
avg = make_averager()
avg(10)

10.0

In [47]:
avg(11)

10.5

In [48]:
avg(12)

11.0

두번째 예제의 `avg()` 함수는 어디에서 `series`를 찾을까? `average` 안에 있는 `series`는 자유 변수(free variable)이다. 이는 지역 범위에 바인딩되어 있지 않은 변수라는 의미이다.

In [49]:
avg.__code__.co_varnames

('new_value', 'total')

In [50]:
avg.__code__.co_freevars

('series',)

`series`에 대한 바인딩은 `avg()` 함수의 `__closure__` 속성에 저장되며, 이는 `avg.__code__.co_freevars`의 이름에 대응된다. 이 항목은 `cell` 객체이며, 이 객체의 `cell_contents` 속성에서 실제값을 찾을 수 있다.

In [51]:
avg.__closure__

(<cell at 0x7ff9fe8b5378: list object at 0x7ff9ff514708>,)

In [52]:
avg.__closure__[0].cell_contents

[10, 11, 12]

클로저는 함수를 정의할 때 존재하던 자유 변수에 대한 바인딩을 유지하는 함수다. 따라서 함수를 정의하는 범위가 사라진 후에 함수를 호출해도 자유 변수에 접근할 수 있다.

함수가 '비전역' 외부 변수를 다루는 경우는 그 함수가 다른 함수 안에 정의된 경우 뿐이다.

# 7.6 nonlocal 선언

앞서 구현한 `make_averager()`는 매번 합계를 재계산하므로 효율적이지 않다. 이를 개선한 다음 함수는 어디가 잘못되었을까?

In [54]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

In [57]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

`averager()` 본체 안에서 `count` 변수에 할당하고 있으므로 `count`를 지역 변수로 만들어 문제가 발생한다. `series`는 가변형 리스트였으므로 문제가 없었다. 그러나 `count = count + 1`과 같은 문장으로 변수를 다시 바인딩하면 암묵적으로 `count`라는 지역 변수를 만든다. `count`가 더이상 자유 변수가 아니므로 클로저에 저장되지 않는다.

이를 해결하기 위해 `nonlocal` 선언이 소개되었다. 변수를 `nonlocal`로 선언하면 함수 안에서 변수에 새로운 값을 할당하더라도 그 변수는 자유 변수로 남는다.

In [64]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

In [65]:
avg = make_averager()
avg(10)

10.0

# 7.7 간단한 데커레이터 구현하기

데커레이트된 함수를 호출할 때마다 시간을 측정하는 데커레이터를 구현해보자.

In [69]:
!cat clockdeco.py

import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked



In [70]:
!cat clockdeco_demo.py

import time
from clockdeco import clock

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
    print('*'*40, 'Calling snooze(.123)')
    snooze(.123)
    print('*'*40, 'Calling factorial(6)')
    print('6! =', factorial(6))


`clock()`은 `factorial()` 함수를 `func` 인수로 받아 `clocked()` 함수를 `factorial`에 할당한다.

In [73]:
!python clockdeco_demo.py

**************************************** Calling snooze(.123)
[0.12327307s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000170s] factorial(1) -> 1
[0.00003449s] factorial(2) -> 2
[0.00005784s] factorial(3) -> 6
[0.00007755s] factorial(4) -> 24
[0.00010590s] factorial(5) -> 120
[0.00013080s] factorial(6) -> 720
6! = 720


In [71]:
import clockdeco_demo
clockdeco_demo.factorial.__name__

'clocked'

따라서 `factorial`이 `clocked`를 참조하는 것을 확인할 수 있다.

이렇게 구현한 `clock()` 데커레이터는 단점이 몇 개 있다. 키워드 인수를 지원하지 않으며, 데커레이트된 함수의 `__name__`과 `__doc__` 속성을 가린다. 이를 `functools.wraps()` 데커레이터를 이용해서 해결할 수 있다.

In [72]:
!cat clockdeco2.py

import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args,**kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        arg_lst = []
        name = func.__name__
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k,w) for k,w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked


In [1]:
!cat clockdeco2_demo.py

import time
from clockdeco2 import clock

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
    print('*'*40, 'Calling snooze(.123)')
    snooze(.123)
    print('*'*40, 'Calling factorial(6)')
    print('6! =', factorial(6))


In [74]:
!python clockdeco2_demo.py

**************************************** Calling snooze(.123)
[0.12323165s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000048s] factorial(1) -> 1
[0.00000978s] factorial(2) -> 2
[0.00001526s] factorial(3) -> 6
[0.00002003s] factorial(4) -> 24
[0.00002813s] factorial(5) -> 120
[0.00003839s] factorial(6) -> 720
6! = 720


In [76]:
import clockdeco2_demo
clockdeco2_demo.factorial.__name__

'factorial'

해결!

# 7.8 표준 라이브러리에서 제공하는 데커레이터

파이썬에서는 메서드를 데커레이트하기 위해 `property()`, `classmethod()`, `staticmethod()` 등 총 3개의 내장 함수를 제공한다. 나중에 살펴보자.

여기서는 `lru_cache()`와 `singledispatch()` 를 알아보자.

## 7.8.1 functools.lru_cache()를 이용한 메모이제이션

`functools.lru_cache()`는 이전에 실행한 값비싼 함수의 결과를 저장함으로써 이전에 사용된 인수에 대해 다시 계산할 필요하게 없게 해주는 메모이제이션(memoization)을 구현한다. 다음 피보나치 수열 생성 예제를 살펴보자.

In [80]:
!cat fibo_demo.py

from clockdemo import clock

@clock
def fibonacci(n):
    if n<2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

if __name__=='__main__':
    print(fibonacci(6))

In [82]:
!python fibo_demo.py

[0.00000024s] fibonacci(0) -> 0
[0.00000031s] fibonacci(1) -> 1
[0.00003866s] fibonacci(2) -> 1
[0.00000015s] fibonacci(1) -> 1
[0.00000019s] fibonacci(0) -> 0
[0.00000017s] fibonacci(1) -> 1
[0.00000751s] fibonacci(2) -> 1
[0.00003632s] fibonacci(3) -> 2
[0.00008208s] fibonacci(4) -> 3
[0.00000014s] fibonacci(1) -> 1
[0.00000014s] fibonacci(0) -> 0
[0.00000016s] fibonacci(1) -> 1
[0.00000749s] fibonacci(2) -> 1
[0.00001360s] fibonacci(3) -> 2
[0.00000013s] fibonacci(0) -> 0
[0.00000016s] fibonacci(1) -> 1
[0.00000746s] fibonacci(2) -> 1
[0.00000014s] fibonacci(1) -> 1
[0.00000021s] fibonacci(0) -> 0
[0.00000017s] fibonacci(1) -> 1
[0.00002089s] fibonacci(2) -> 1
[0.00002796s] fibonacci(3) -> 2
[0.00004227s] fibonacci(4) -> 3
[0.00006286s] fibonacci(5) -> 5
[0.00015219s] fibonacci(6) -> 8
8


이는 계산 낭비가 엄청나다. 이를 `lru_cache()`를 이용하여 개선해보자.

In [83]:
!cat fibo_demo_lru.py

import functools
from clockdeco import clock

@functools.lru_cache()
@clock
def fibonacci(n):
    if n<2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

if __name__=='__main__':
    print(fibonacci(6))

In [84]:
!python fibo_demo_lru.py

[0.00000029s] fibonacci(0) -> 0
[0.00000023s] fibonacci(1) -> 1
[0.00004025s] fibonacci(2) -> 1
[0.00000036s] fibonacci(3) -> 2
[0.00006736s] fibonacci(4) -> 3
[0.00000032s] fibonacci(5) -> 5
[0.00009442s] fibonacci(6) -> 8
8


`lru_cache()`는 다음 두 개의 인수를 지닌다.
* maxsize : 얼마나 많은 호출을 저장할지 결정
* typed : True일 때 인수의 자료형이 다르면 결과를 따로 저장

`lru_cache()`에 의해 데커레이트된 함수의 인수는 모두 해시 가능해야 한다.

## 7.8.2 단일 디스패치를 이용한 범용 함수

웹 어플리케이션을 디버깅하는 도구를 만들고 있다고 가정하자.

In [85]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

In [86]:
html.escape?

Signature: html.escape(s, quote=True)
Docstring:
Replace special characters "&", "<" and ">" to HTML-safe sequences.
If the optional flag quote is true (the default), the quotation mark
characters, both double quote (") and single quote (') characters are also
translated.
File:      /usr/lib/python3.7/html/__init__.py
Type:      function


일부 자료형에 대해 다음과 같이 고유한 코드를 생성하도록 이 코드를 확장하려고 한다.

* str : 개행 문자를 `<br>\n`으로 대체하고 `<pre>` 대신 `<p>` 태그를 사용한다.
* int : 숫자를 10진수와 16진수로 보여준다.
* list : 각 항목을 자료형에 따라 포맷한 HTML 리스트를 출력한다.

이는 `if/elif/elif`와 특화함수 등을 이용하여 구현할 수 있지만 시간이 지나면서 관리가 너무 어려워진다. 

이를 해결하기 위해 `functools.singledispatch()` 데커레이터가 파이썬 3.4에서 새로 소개되었다. 일반 함수를 `singledispatch()`로 데커레이트하면 이 함수는 범용 함수(generic function)이 되어, 첫 번째 인수의 자료형에 따라 서로 다른 방식으로 연산을 수행하게 된다.

In [92]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str) # 각각의 특화된 함수는 @<기반_함수>.register(<객체형>)으로 데커레이트
def _(text): # 특화된 함수의 이름은 필요없음
    content = html.escape(text).replace('\n','<br>\n')
    return '<p>{}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>{}</li>\n</ul>'.format(inner)

가능하면 `int`나 `list`와 같은 구상 클래스보다 `numbers.Integral`이나 `abc.MutableSequence`와 같은 추상 베이스 클래스를 처리하도록 특화된 함수를 등록하는 것이 좋다.

`singledispatch` 메커니즘은 특화된 함수를 시스템 어디엔, 어느 모듈에나 등록할 수 있다는 장점이 있다.

# 7.9 누적된 데커레이터

하나의 함수 `f()`에 두 데커레이터 `@d1`과 `@d2`를 차례대로 적용하면 결과는 `f = d1(d2(f))`와 같다.

# 7.10 매개변수화된 데커레이터

다른 인수를 받는 데커레이터를 어떻게 만들 수 있는지에 대해 알아보자.

In [93]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
print('running main()')
print('registry ->', registry)
f1()

running register(<function f1 at 0x7ff9fec522f0>)
running main()
registry -> [<function f1 at 0x7ff9fec522f0>]
running f1()


## 7.10.1 매개변수화된 등록 데커레이터

`register()`가 등록하는 함수를 활성화 혹은 비활성화하기 쉽게 만들기 위해 선택 인수 `active`를 도입하자.

In [3]:
registry = set()

def register(active=True):
    def decorate(func):
        print('running register(active=%s)->decorate(%s)' % (active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
            
        return func
    return decorate

@register(active=False)
def f1():
    print('running f1()')
    
@register()
def f2():
    print('running f2()')

def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x7f98ca15e1e0>)
running register(active=True)->decorate(<function f2 at 0x7f98ca15e400>)


In [95]:
registry

{<function __main__.f2()>}

## 7.10.2 매개변수화된 clock 데커레이터

이 절에서는 `clock()` 데커레이터를 수정하여, 사용자가 포맷 문자열을 전달해서 데커레이트된 함수가 출력할 문자열을 설정한다.

In [96]:
!cat clockdeco_param.py

import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT): # 매개변수화된 데커레이터 팩토리
    def decorate(func): # 실제 데커레이터
        def clocked(*_args): #데커레이트된 함수를 래핑
            t0 = time.time()
            _result = func(*_args) # 함수의 실제 연산 결과
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result) # _result를 문자화
            print(fmt.format(**locals())) # fmt가 clocked()의 지역 변수를 모두 참조
            return _result # clocked는 데커레이트된 함수를 대체하므로, 원래 함수가 반환하는 값을 반환
        return clocked
    return decorate


if __name__=='__main__':
    
    @clock()
    def snooze(seconds):
        time.sleep(seconds)
        
    for i in range(3):
        snooze(.123)

In [97]:
!python clockdeco_param.py

[0.12324476s] snooze(0.123) -> None
[0.12322211s] snooze(0.123) -> None
[0.12327051s] snooze(0.123) -> None


다른 예제를 살펴보자.

In [98]:
!cat clockdeco_param_demo1.py

import time
from clockdeco_param import clock

@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

In [99]:
!python clockdeco_param_demo1.py

snooze: 0.12325072288513184s
snooze: 0.1232154369354248s
snooze: 0.1232461929321289s


In [100]:
!cat clockdeco_param_demo2.py

import time
from clockdeco_param import clock

@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

In [101]:
!python clockdeco_param_demo2.py

snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s
snooze(0.123) dt=0.123s


# 끗!